Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Catch up, if needed
- [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2/portfolio-project/ds6), then choose your dataset, and [submit this form](https://forms.gle/nyWURUg65x1UTRNV9), due yesterday at 3:59pm Pacific.
- Submit predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.) The competition closes today at 3:59pm. Every student should make at least one submission that scores at least 60% accuracy (above the majority class baseline).

## Assignment
- [X] Continue to participate in our Kaggle challenge. 
- [X] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [X] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_ The competition closes today at 3:59pm.
- [X] Add comments and Markdown to your notebook. Clean up your code.
- [X] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Try combining xgboost early stopping, cross-validation, & hyperparameter optimization, with [the "original" (non scikit-learn) xgboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.cv).
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

#### Try stacking multiple submissions!

Here's some code you can use:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # eli5, version >= 0.9
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders eli5 pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: eli5 in /usr/local/lib/python3.6/dist-packages (0.9.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

In [0]:
X_train, X_val, y_train, y_val = train_test_split(train.drop('status_group', axis = 'columns'),train.status_group, test_size = .25, stratify = train.status_group)

In [0]:
# Defining feature engineering function
import datetime
import numpy as np

def engineer(df):
  df = df.copy()
#   Making age variable
  df.date_recorded = pd.to_datetime(df.date_recorded)
  df.construction_year.replace(0,np.NaN, inplace = True)
  mean_year = np.nanmean(df.construction_year)
  df.construction_year.replace(np.NaN,mean_year, inplace = True)
  df['age'] = df.date_recorded.dt.year - df.construction_year
#   Adding day, month, year, and day of week features
  df['day_recorded'] = df.date_recorded.dt.day
  df['month_recorded'] = df.date_recorded.dt.month
  df['year_recorded'] = df.date_recorded.dt.year
  df['day_of_week_recorded'] = df.date_recorded.dt.dayofweek
  df.drop('date_recorded', axis = 'columns', inplace = True)
#   putting nan values where zeros are but shouldn't be
  cols_with_zeros = ['longitude', 'latitude', 
                       'gps_height', 'population']
  for col in cols_with_zeros:
    df[col] = df[col].replace(0, np.nan)
    df[col+'_MISSING'] = df[col].isnull()
# Drop duplicate columns
  duplicates = ['quantity_group', 'payment_type']
  df = df.drop(columns=duplicates)
# Drop recorded_by (never varies) and id (always varies, random)
  unusable_variance = ['recorded_by', 'id']
  df = df.drop(columns=unusable_variance)
#   Making region code categorical instead of numeric
  df['region_code'] = pd.Categorical(df.region_code)
  return df

In [0]:
# applying engineer function
X_train = engineer(X_train)
X_val = engineer(X_val)

In [0]:
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier


# pipeline = make_pipeline(
#     ce.OrdinalEncoder(), 
#     SimpleImputer(strategy='median'), 
#     RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
# )

# # Fit on train, score on val
# pipeline.fit(X_train, y_train)
# print('Validation Accuracy', pipeline.score(X_val, y_val))

pipe = make_pipeline(
          ce.OrdinalEncoder()
          , IterativeImputer()
          , RandomForestClassifier(n_estimators = 2000, min_samples_leaf = 9, max_depth = 33, n_jobs = -1)
)




In [27]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['funder', 'installer', 'wpt_name',
                                      'basin', 'subvillage', 'region',
                                      'region_code', 'lga', 'ward',
                                      'public_meeting', 'scheme_management',
                                      'scheme_name', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'water_quality', 'quality_g...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=33,
                                        max_features='auto',
                                        max_leaf

In [28]:
from sklearn.metrics import accuracy_scor
y_pred = pipe.predict(X_val)
print('Validation Accuracy', accuracy_score(y_pred, y_val))

Validation Accuracy 0.7952188552188553


In [17]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'randomforestclassifier__n_estimators':range(500,5000,500)
    , 'randomforestclassifier__max_depth':range(10,50,5)
    , 'randomforestclassifier__max_features':range(5, len(X_train.columns)+1)
    , 'randomforestclassifier__min_samples_leaf':range(1,50,5)
#     , 'targetencoder_smoothing':range(0,20,3)
}

search = RandomizedSearchCV(pipe
                            ,param_distributions = param_dist
                            ,n_iter = 100
                            ,cv = 3
                            ,scoring = 'accuracy'
                            ,verbose = 10
                            ,return_train_score = True
                            ,n_jobs = -1
                           )

search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 10.7min


KeyboardInterrupt: ignored

In [18]:
print('Best parameters: ', search.best_params_)
print('Best score: ',search.best_score_)

AttributeError: ignored

In [0]:
from sklearn import metrics
sorted(metrics.SCORERS.keys())